## 1.导入包

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import re
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

np.set_printoptions(threshold=np.inf)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
"""
网格搜索(Grid Search)和Optuna，找到模型的最佳超参数组合
网格搜索适用于超参数空间较小、离散且较少的情况，而Optuna适用于超参数空间较大、连续或离散且较多的情况
下面要做的事情：
1.换新的面心值标签，现在数据过拟合，训练集下降但是测试集上升或者波动
2.考虑正则化或者假如droput层来防止过拟合
3.考虑数据预处理中采用数据标准化，让数据均匀分布
4.用不用考虑损失函数，学习率,epoch,adam优化器及其四个参数，的修改，模型用不用再添加几层让模型变复杂些（batch-size越大，训练越快，不影响准确率）
5.早停法（Early Stopping）：在训练过程中监控验证集上的性能，一旦性能停止改善，在一定epoch后停止训练，并保存模型，以防止过拟合。可以参照外国那案例
6.数据集的比例，不一定4：1，也可以95：5，当数据集足够大时，这样可以增加训练集数量
"""

'\n网格搜索(Grid Search)和Optuna，找到模型的最佳超参数组合\n网格搜索适用于超参数空间较小、离散且较少的情况，而Optuna适用于超参数空间较大、连续或离散且较多的情况\n下面要做的事情：\n1.换新的面心值标签，现在数据过拟合，训练集下降但是测试集上升或者波动\n2.考虑正则化或者假如droput层来防止过拟合\n3.考虑数据预处理中采用数据标准化，让数据均匀分布\n4.用不用考虑损失函数，学习率,epoch,adam优化器及其四个参数，的修改，模型用不用再添加几层让模型变复杂些（batch-size越大，训练越快，不影响准确率）\n5.早停法（Early Stopping）：在训练过程中监控验证集上的性能，一旦性能停止改善，在一定epoch后停止训练，并保存模型，以防止过拟合。可以参照外国那案例\n6.数据集的比例，不一定4：1，也可以95：5，当数据集足够大时，这样可以增加训练集数量\n'

## 2.加载数据

"""
1.功能：
通过加载data和label文件，然后继续训练和预测。
定义了一个6层卷积神经网络模型。每个卷积层后面跟着一个 ReLU 激活函数。第七层只有卷积，没有relu。
输入数据n*64*64*2,这里的一个样本64*64可以看成一个图片格式（在此次任务中是速度，两者类似）
输出是n*64*64*4
"""
"""txt保存为numpy格式发现可以减少存储大小，约缩小成1/4
5.9G	./all_data.npy
12G	./all_label.npy
27G	./data_64x64x2.txt
53G	./label_a_2x64x65x2.txt
"""

In [3]:
# 直接加载npy文件为numpy格式
all_data = np.load('./data/all_data.npy')
# #直接加载npy文件为numpy格式
all_label = np.load('./data/all_centerFace_label.npy')

all_data = torch.tensor(all_data).float()
all_label = torch.tensor(all_label).float()


In [4]:
#神经网络模型
class Net(nn.Module):
    def __init__(self, num_output_channels):
        super(Net, self).__init__()

        # 第一层卷积，使用64个大小为3x3的卷积核，输入数据的shape为2x64x64，使用ReLU激活函数。
        # 输入通道数为2，输出通道数恒为64
        self.conv1 = nn.Conv2d(2, 64, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()

        # 第二层卷积，使用64个大小为3x3的卷积核，使用ReLU激活函数
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()

        # 第三层卷积，使用64个大小为3x3的卷积核，使用ReLU激活函数
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()

        # 第四层卷积，使用64个大小为3x3的卷积核，使用ReLU激活函数
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()

        # 第五层卷积，使用64个大小为3x3的卷积核，使用ReLU激活函数
        self.conv5 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu5 = nn.ReLU()

        # 第六层卷积，使用64个大小为3x3的卷积核，使用ReLU激活函数
        self.conv6 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu6 = nn.ReLU()

        # 第七层卷积，使用64个大小为3x3的卷积核，使用ReLU激活函数,输出数据为4*64*64
        # 输入通道数为64，输出通道数恒为4
        self.conv7 = nn.Conv2d(64, num_output_channels, kernel_size=3, stride=1, padding=1)



    def forward(self, x):
        x = self.conv1(x)  # x:torch.Size([10, 2, 64, 64])
        x = self.relu1(x)

        x = self.conv2(x)  # x:torch.Size([10, 64, 64, 64])
        x = self.relu2(x)

        x = self.conv3(x)  # x:  torch.Size([10, 64, 64, 64])
        x = self.relu3(x)

        x = self.conv4(x)
        x = self.relu4(x)

        x = self.conv5(x)
        x = self.relu5(x)

        x = self.conv6(x)
        x = self.relu6(x)

        x = self.conv7(x)

        # 输出数据shape: torch.Size([10, 4, 64, 64]),10是batch_size大小
        return x



## 3.加载模型

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 加载模型
model = torch.load('./model/model_origin_100.pth', map_location=device)# from torchsummary import summary
# # 将模型移动到适当的设备
model = model.to(device)


## 4.预测

In [6]:
# 设置模型为评估模式
model.eval()

# 输入数据进行预测
input_data =all_data[-10]  # 你的输入数据
print("all_data[0].shape:",input_data.shape)

#调整输入input的维度顺序,作为E，用于下面change_Label_to_a中(E-A)/(B-A)得到a值
matrix_64 = input_data.cpu()
matrix_64 = matrix_64.permute(1,2,0)  
print(matrix_64.shape)


# 转换为四维
input_data = input_data.unsqueeze(0)#用实际数据，数据格式为(1,2, 64, 64)，不能为2x64x64
# input_data = torch.randn(1,2, 64, 64)
print(input_data.shape)
input_tensor = input_data.to(device)

with torch.no_grad():
    output = model(input_tensor)##如果报错的话需要把网络的设计加上，里面涉及model(input)

# 打印预测结果
print(output)

all_data[0].shape: torch.Size([2, 64, 64])
torch.Size([64, 64, 2])
torch.Size([1, 2, 64, 64])
tensor([[[[2.4147, 2.6336, 2.6039,  ..., 2.6414, 2.8297, 2.6371],
          [2.7359, 2.9315, 2.7377,  ..., 2.8831, 3.0777, 3.0277],
          [2.8565, 2.9522, 2.9528,  ..., 3.0358, 2.9106, 2.9598],
          ...,
          [3.0433, 3.2143, 3.2400,  ..., 2.6568, 2.8410, 2.9792],
          [2.8984, 3.1541, 3.0952,  ..., 2.8528, 2.9644, 2.9886],
          [2.3506, 2.9378, 3.0302,  ..., 2.7801, 2.9479, 2.7217]],

         [[0.6954, 0.6169, 0.4632,  ..., 1.6724, 1.4816, 1.1916],
          [0.9007, 0.6778, 0.3545,  ..., 1.5664, 1.3687, 1.3655],
          [1.0699, 0.7895, 0.5898,  ..., 1.5141, 1.3430, 1.3209],
          ...,
          [1.3977, 0.8905, 0.5173,  ..., 1.5435, 1.4066, 1.5906],
          [1.1747, 0.8391, 0.5226,  ..., 1.5055, 1.3806, 1.5327],
          [0.9317, 0.9692, 0.7692,  ..., 1.5999, 1.5031, 1.3977]],

         [[3.1068, 2.9709, 2.9812,  ..., 2.3603, 2.8287, 2.6615],
          [2.7

In [7]:
all_label[-10]

tensor([[[2.7622, 2.7929, 2.8339,  ..., 2.7490, 2.7284, 2.7387],
         [2.8790, 2.8889, 2.9158,  ..., 2.9171, 2.9076, 2.8882],
         [3.0331, 3.0480, 3.0696,  ..., 3.0109, 3.0071, 3.0193],
         ...,
         [2.9981, 3.1974, 3.3818,  ..., 2.4901, 2.6205, 2.7967],
         [2.7969, 2.9431, 3.0806,  ..., 2.5270, 2.5761, 2.6679],
         [2.7101, 2.7851, 2.8750,  ..., 2.6107, 2.6347, 2.6616]],

        [[0.9897, 0.7952, 0.5879,  ..., 1.4945, 1.3351, 1.1696],
         [0.9734, 0.7663, 0.5483,  ..., 1.5202, 1.3575, 1.1713],
         [0.9716, 0.7526, 0.5196,  ..., 1.5206, 1.3549, 1.1733],
         ...,
         [1.3045, 1.1354, 0.9128,  ..., 1.5814, 1.5034, 1.4197],
         [1.1302, 0.9680, 0.7664,  ..., 1.5156, 1.3909, 1.2654],
         [1.0275, 0.8502, 0.6546,  ..., 1.4898, 1.3525, 1.1947]],

        [[2.7404, 2.8140, 2.9748,  ..., 3.2381, 2.9728, 2.7836],
         [2.7895, 2.8467, 2.9836,  ..., 3.4670, 3.1368, 2.9018],
         [2.8532, 2.8868, 3.0103,  ..., 3.6718, 3.2788, 3.

## 5.转换格式(label转为最终的weights)

In [8]:
#将label面心值转为a值
def change_Label_to_a(all_vertical_edge_centers,all_horizontal_edge_centers):
    a_vertical = np.zeros((64, 65, 2))
    # a_vertical = np.random((64, 65, 2))
    a_horizontal = np.zeros((64, 65, 2))
    # a_horizontal = np.random.random((64, 65, 2))


    # 21. 求a:   横着的边，分两种情况，边缘（对称的）和非边缘的边.这里matrix_64要行列互换，因为横着时面心值是一列一列求得，竖着时是一行一行求的。
    for i in range(64):
        for j in range(65):
            if j == 0 or j == 64:
                # a_horizontal[i, j] = (all_horizontal_edge_centers[i, 0] - matrix_64[i, 0]) / (matrix_64[i, 63] - matrix_64[i, 0])#换之前
                a_horizontal[i, j] = (all_horizontal_edge_centers[i, 0] - matrix_64[0, i]) / (
                    matrix_64[63, i] - matrix_64[0, i])
            else:
                # aA=(1-a)B=E   a = (E-B)/(A-B) 其中：A为matrix_64[i,j]，B为matrix_64[i,j+1]
                # a_horizontal[i, j] = (all_horizontal_edge_centers[i, j] - matrix_64[i, j]) / (matrix_64[i, j-1] - matrix_64[i, j])
                a_horizontal[i, j] = (all_horizontal_edge_centers[i, j] - matrix_64[j, i]) / (
                    matrix_64[j - 1, i] - matrix_64[j, i])

    # 22. 求a:   竖着的边，分两种情况，边缘（对称的）和非边缘的边
    for i in range(64):
        for j in range(65):
            if j == 0 or j == 64:
                a_vertical[i, j] = (all_vertical_edge_centers[i, 0] - matrix_64[i, 0]) / (
                    matrix_64[i, 63] - matrix_64[i, 0])
            else:
                # aA=(1-a)B=E   a = (E-B)/(A-B) 其中：A为matrix_64[i,j]，B为matrix_64[i,j+1]
                # 2.错误matrix_64[i, j-1]) / (改成matrix_64[i, j]) / (
                a_vertical[i, j] = (all_vertical_edge_centers[i, j] - matrix_64[i, j]) / (
                    matrix_64[i, j - 1] - matrix_64[i, j])

    # # 若最终a对应的矩阵里面出现无穷，则将其替换为0.5.解决了分母为0的问题
    # a_vertical[np.isinf(a_vertical)] = 0.5
    # a_horizontal[np.isinf(a_horizontal)] = 0.5
    #这里64x65x2截成64x64x2,因为边框对称时值相同
    
    a_vertical = torch.tensor(a_vertical[:,:64,:])
    a_horizontal = torch.tensor(a_horizontal[:,:64,:])

    print(a_vertical.shape)
    print(a_horizontal.shape)
    
    return  a_vertical,a_horizontal

In [9]:
#返回一个列表，里面嵌套两个子列表，第一个子列表存放的是内部的a值，第二个子列表存放的是边框的a值，
#且顺序为上（左到右），下（左到右），左（下到上），右（下到上）
def conversion_format(output):
    # 调整output的维度顺序
    output = output.permute(0,2,3,1)  #torch.Size([1, 64, 64, 4])
    print(output.shape)
    output = output[0]#torch.Size([64, 64, 4])
    print(output.shape)
    # print(output)

    #将输出output拆成两个面心值
    all_vertical_edge_centers = output[:, :, 0:2].cpu()
    all_horizontal_edge_centers = output[:, :, 2:4].cpu()
#     print(all_vertical_edge_centers.shape)
#     print(all_horizontal_edge_centers.shape)

    
    a_vertical,a_horizontal  = change_Label_to_a(all_vertical_edge_centers,all_horizontal_edge_centers)    
    
    # 将前两个元素相加除以二得到一个元素(x+y/)2
    avg_vertical = (a_vertical[:, :, 0] + a_vertical[:, :, 1]) / 2
    # 将后两个元素相加除以二得到另一个元素
    avg_horizontal = (a_horizontal[:, :, 0] + a_horizontal[:, :, 1]) / 2

    # 重新组合成新的形状为(64, 64, 2)的张量
    new_avg_a_output = torch.stack([avg_vertical, avg_horizontal], dim=2)
    # 打印转换后的数据形状
#     print(new_avg_a_output.shape)

    #返回两个求完平均的面心值,包括两个64*64矩阵，矩阵是求完平均后的a值，一个竖着的，一个横着的
    vertical_1d = new_avg_a_output[:, :, 0]
    horizontal_1d = new_avg_a_output[:, :,1]
#     print(vertical_1d.shape)
#     print(horizontal_1d.shape)
    
    
    border = []#存所有边框，四个边框
    left_border=[]#存左边框
    bottom_border = []#存下边框
    inner = []#存内部的面心值

    #下面将2个64x64面心值变换格式，返回指定的格式result
    for i in range(len(vertical_1d)): #两个for循环等价于for i in range(64):
        for j in range(len(vertical_1d[i])):
            if j ==0:#j=0添加边框
                #添加左边框
                left_border.append(vertical_1d[i][0])
                #添加下边框
                bottom_border.append(horizontal_1d[i][0])
            else: 
                if i !=  63:#当竖着的最后一行时，上面没有对应的横着的
                    inner.append(vertical_1d[i][j])#竖着的
    #                 print("{j-1},{i+1}",j-1,i+1)
                    inner.append(horizontal_1d[j-1][i+1])   #再横着的   
                    if j == 63:#如果j=63的话，还需要再加入最后一列的横着的边
                        inner.append(horizontal_1d[63][i+1])#当i=63,横着的加最后一列 
                else:#if i ==63 :
                    inner.append(vertical_1d[63][j])#当i=63,inner最后添加竖着的一行竖线

    inner = [tensor.cpu().numpy().tolist() for tensor in inner]#将一维列表里面的tensor元素转为numpy格式，并返回cpu版本
    # print(inner)
    four_border = [bottom_border,bottom_border,left_border,left_border]#顺序是上（左到右），下（左到右），左（下到上），右（下到上）
    border = [item.numpy().tolist() for sublist in four_border for item in sublist]
    result = [inner,border]
    print(len(result))
    return result



In [10]:
result  = conversion_format(output)

torch.Size([1, 64, 64, 4])
torch.Size([64, 64, 4])
torch.Size([64, 64, 2])
torch.Size([64, 64, 2])
2


In [11]:
print(len(result[0]))
print(len(result[1]))
print(8064+64*2)

8064
256
8192


In [12]:
result

[[2.230677217245102,
  -1.147687405347824,
  2.655281774699688,
  -0.46663153171539307,
  1.4663721695542336,
  0.7253761291503906,
  -0.33424731716513634,
  -4.214919090270996,
  -1.0538312494754791,
  -2.734604597091675,
  -0.308481365442276,
  -5.476034641265869,
  -0.0957077145576477,
  -1.1556915044784546,
  0.27969203144311905,
  16.706998348236084,
  0.5397417321801186,
  9.836561441421509,
  0.6400988399982452,
  6.772923707962036,
  0.6267368420958519,
  2.878944754600525,
  0.7530100494623184,
  3.5030141472816467,
  0.7428830862045288,
  4.4509687423706055,
  0.6705436259508133,
  -18.811028480529785,
  -3.93136990070343,
  1.7208044528961182,
  3.170951522886753,
  1.338860273361206,
  1.2047257497906685,
  0.44052720069885254,
  0.4750605821609497,
  -8.330178499221802,
  0.27081096172332764,
  -7.614922046661377,
  0.6470327079296112,
  -6.430557012557983,
  1.0715023269876838,
  -10.10556936264038,
  1.468636691570282,
  -5.234441518783569,
  8.92196923494339,
  16.20060

In [13]:
count = sum(1 for num in result[0] if 0 < num < 1)
print(count)
print(count/len(result[0]))

1753
0.2173859126984127
